In [3]:
import pandas as pd
import numpy as np
!pip install liwc


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from nltk.corpus import stopwords
import collections
import liwc
from collections import Counter

In [8]:
df = []

In [8]:
data_files = ['schizophrenia.csv','fitness.csv','jokes.csv','meditation.csv','parenting.csv','relationships.csv','teaching.csv']
for file in data_files:
    data = pd.read_csv(file)
    df.append(data)
df = pd.concat(df)

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.dropna()

In [ ]:
df['Label'].value_counts()

In [ ]:
df['message'] = df['Post Text']

In [ ]:
import seaborn as sns
sns.countplot(x="Label", data = df)

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

In [ ]:
comment_words = ''
stopwords = set(STOPWORDS)

In [ ]:
for val in df.message:
     
    # typecaste each val to string
    val = str(val)
 
    # split the value
    tokens = val.split()
     
    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
     
    comment_words += " ".join(tokens)+" "

In [ ]:
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(comment_words)

In [ ]:
# plot the WordCloud image                      
plt.figure(figsize = (12, 12), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
 
plt.show()

In [ ]:
X = df['message']
y = df['Label']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

cv = CountVectorizer()
X = cv.fit_transform(X) # Fit the Data

In [ ]:
parse, category_names = liwc.load_token_parser('LIWC2007_English100131.dic')

In [ ]:
#LIWC Features Extraction
liwc =[] 
for item in df.message:
    gettysburg_counts = list(collections.Counter(category for message in item for category in parse(message) if category == 'schizophrenia').items())
    liwc.append(gettysburg_counts)
liwc_ = np.array(liwc)


# Machine Learning

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RandomForest = RandomForestClassifier(n_estimators=10, random_state=0)
RandomForest.fit(X, y) 
y_pred = RandomForest.predict(X)
print(confusion_matrix(y,y_pred))
print(classification_report(y,y_pred))
print(accuracy_score(y, y_pred))
RF = accuracy_score(y, y_pred)

# Support Vector Machine

In [ ]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(X, y) 
y_pred = clf.predict(X)
print(confusion_matrix(y,y_pred))
print(classification_report(y,y_pred))
print(accuracy_score(y, y_pred))
SVM = accuracy_score(y, y_pred)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X, y) 
y_pred = clf.predict(X)
print(confusion_matrix(y,y_pred))
print(classification_report(y,y_pred))
print(accuracy_score(y, y_pred))
LR = accuracy_score(y, y_pred)

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X, y) 
y_pred = clf.predict(X)
print(confusion_matrix(y,y_pred))
print(classification_report(y,y_pred))
print(accuracy_score(y, y_pred))
NB = accuracy_score(y, y_pred)

# Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn.svm import SVC

estimators = []
model1 = LogisticRegression()
estimators.append(('logistic', model1))
model2 = RandomForestClassifier()
estimators.append(('cart', model2))
model3 = SVC()
estimators.append(('svm', model3))

# create the ensemble model
ensemble = VotingClassifier(estimators)
warnings.simplefilter("ignore")

print(ensemble)
ensemble.fit(X, y)

y_pred = ensemble.predict(X)
print(confusion_matrix(y,y_pred))
print(classification_report(y,y_pred))
print(accuracy_score(y, y_pred))

E= accuracy_score(y, y_pred)

# Stacking Classifier

In [ ]:
from sklearn.ensemble import StackingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

estimators = [('AdaBoost', AdaBoostClassifier(n_estimators=100, random_state=0)),
              ('CatBoost',CatBoostClassifier(verbose=0, n_estimators=100))]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

In [ ]:
clf.fit(X, y) 
y_pred = clf.predict(X)
print(confusion_matrix(y,y_pred))
print(classification_report(y,y_pred))
print(accuracy_score(y, y_pred))
Stack = accuracy_score(y, y_pred)

# Deep Learning

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, callbacks
from tensorflow.keras import Model, Sequential

from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import re
import string

In [ ]:
# Encoded Sentiment columns

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['Label'] = encoder.fit_transform(df['Label'])

TypeError: ignored

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
num_classes = df.Label.nunique()
print(num_classes)

In [ ]:
def cleaning_text(text):
    stop_words = stopwords.words("english")

    text = re.sub(r'http\S+', " ", text)    # remove urls
    text = re.sub(r'@\w+',' ',text)         # remove mentions
    text = re.sub(r'#\w+', ' ', text)       # remove hastags
    text = re.sub('r<.*?>',' ', text)       # remove html tags
    
    # remove stopwords 
    text = text.split()
    text = " ".join([word for word in text if not word in stop_words])

    for punctuation in string.punctuation:
        text = text.replace(punctuation, "")
    
    return text

In [ ]:
df.info()

In [ ]:
df['text'] = df['text'].apply(lambda x: cleaning_text(x))

In [ ]:
# Maximum sentence length
max_len_words = max(list(df['text'].apply(len)))
print(max_len_words)

In [ ]:
def tokenizer(x_train, y_train, max_len_word):
    # because the data distribution is imbalanced, "stratify" is used
    X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                      test_size=.2, shuffle=True, 
                                                      stratify=y_train, random_state=0)

    # Tokenizer
    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(X_train)
    sequence_dict = tokenizer.word_index
    word_dict = dict((num, val) for (val, num) in sequence_dict.items())

    # Sequence data
    train_sequences = tokenizer.texts_to_sequences(X_train)
    train_padded = pad_sequences(train_sequences,
                                 maxlen=max_len_word,
                                 truncating='post',
                                 padding='post')
    
    val_sequences = tokenizer.texts_to_sequences(X_val)
    val_padded = pad_sequences(val_sequences,
                                maxlen=max_len_word,
                                truncating='post',
                                padding='post', )
    
    print(train_padded.shape)
    print(val_padded.shape)
    print('Total words: {}'.format(len(word_dict)))
    return train_padded, val_padded, y_train, y_val, word_dict

X_train, X_val, y_train, y_val, word_dict = tokenizer(df.text, df.Label, 100)

# DNN 

In [ ]:
import tensorflow as tf
model = Sequential([
    layers.Embedding(5000, 100, input_length=100),
    layers.SimpleRNN(64, return_sequences=True, recurrent_dropout=0.4),
    layers.GlobalAveragePooling1D(),    # or layers.Flatten()
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
trained = model.fit(X_train, y_train,
                    epochs=10, 
                    validation_data=(X_val, y_val),
                    shuffle=True)

In [ ]:
plt.plot(trained.history['accuracy'])
plt.plot(trained.history['val_accuracy'])
plt.title('Accuracy Curves')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(trained.history['loss'])
plt.plot(trained.history['val_loss'])
plt.title('Loss Curves')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
rnn = trained.history['val_accuracy']

# CNN

In [ ]:
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam, SGD
def build_cnn_model():
    model = Sequential()
    
    model.add(Embedding(5000, 100, input_length=100))


    model.add(Conv1D(64, 2, padding='same', activation='relu'))
    model.add(MaxPooling1D(2))
    #model.add(MaxPooling1D(2))
    
    model.add(Flatten())
    
    model.add(Dense(units=1024,activation="relu"))
    model.add(Dense(units=512,activation="relu"))
    
    model.add(Dense(units=num_classes,activation="softmax"))
    
    optimizer = Adam(lr=0.000055,beta_1=0.9,beta_2=0.999)
    
    model.compile(optimizer=optimizer,metrics=["accuracy"],loss=tf.keras.losses.SparseCategoricalCrossentropy())
    return model

In [ ]:
cnn_model = build_cnn_model()
cnn_history = cnn_model.fit(X_train, y_train,
                    epochs=10, 
                    validation_data=(X_val, y_val),
                    shuffle=True)

In [ ]:
cnn = cnn_history.history['val_accuracy']

In [ ]:
plt.plot(cnn_history.history['accuracy'])
plt.plot(cnn_history.history['val_accuracy'])
plt.title('Accuracy Curves')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(cnn_history.history['loss'])
plt.plot(cnn_history.history['val_loss'])
plt.title('Loss Curves')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# GNN - GRU

In [ ]:
import tensorflow as tf
model = Sequential([
    layers.Embedding(5000, 100, input_length=100),
    layers.GRU(64, return_sequences=True, recurrent_dropout=0.4),
    layers.GlobalAveragePooling1D(),    # or layers.Flatten()
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
trained = model.fit(X_train, y_train,
                    epochs=10, 
                    validation_data=(X_val, y_val),
                    shuffle=True)

In [ ]:
gru = trained.history['val_accuracy']

In [ ]:
plt.plot(trained.history['accuracy'])
plt.plot(trained.history['val_accuracy'])
plt.title('Accuracy Curves')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(trained.history['loss'])
plt.plot(trained.history['val_loss'])
plt.title('Loss Curves')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# LSTM

In [ ]:
import tensorflow as tf
model = Sequential([
    layers.Embedding(5000, 100, input_length=100),
    layers.LSTM(64, return_sequences=True, recurrent_dropout=0.4),
    layers.GlobalAveragePooling1D(),    # or layers.Flatten()
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
trained = model.fit(X_train, y_train,
                    epochs=10, 
                    validation_data=(X_val, y_val),
                    shuffle=True)

In [ ]:
lstm = trained.history['val_accuracy']

In [ ]:
plt.plot(trained.history['accuracy'])
plt.plot(trained.history['val_accuracy'])
plt.title('Accuracy Curves')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(trained.history['loss'])
plt.plot(trained.history['val_loss'])
plt.title('Loss Curves')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# CNN + LSTM

In [ ]:
model = Sequential()
model.add(Embedding(5000, 100, input_length=100))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(num_classes, activation='sigmoid'))
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
print(model.summary())
filepath="weights_best_cnn.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max',save_weights_only=True)
callbacks_list = [checkpoint]


In [ ]:
trained = model.fit(X_train, y_train,
                    epochs=10, 
                    validation_data=(X_val, y_val),
                    shuffle=True)

In [ ]:
ense = trained.history['val_accuracy']

In [ ]:
plt.plot(trained.history['accuracy'])
plt.plot(trained.history['val_accuracy'])
plt.title('Accuracy Curves')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(trained.history['loss'])
plt.plot(trained.history['val_loss'])
plt.title('Loss Curves')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
accuracies = [RF, SVM, LR, NB,E,Stack,rnn[9],cnn[9],gru[9],lstm[9],ense[9]]

In [ ]:
for i in range(0, len(accuracies)):
    accuracies[i] = accuracies[i]*100
    print(accuracies[i])

In [ ]:
models = ['Random Forest', 'SVM', 'Logistic Regression','MNB','Voting Classifier','Stacking Boosting','DNN','CNN','GNN','LSTM','CNN+LSTM']

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(models,accuracies)
plt.xlabel("For First Dataset")
plt.ylabel("Accuracy (in %)")
plt.show()

In [ ]:
import joblib
filename = 'model.sav'
joblib.dump(RandomForest, filename)